In [1]:
import os

In [2]:
%pwd

'd:\\Machine Learning\\Projects\\Text-Summarizer\\Attempt\\Text-Summarizer\\pre-tests'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Machine Learning\\Projects\\Text-Summarizer\\Attempt\\Text-Summarizer'

In [5]:
# define entity here
from dataclasses import dataclass
from pathlib import Path

#its a data class where one can mention their variables
# pull variables from config.yaml and type in the data types
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [11]:
#define configuration for data ingestion here
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create directory for datasets
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [12]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-15 00:37:24,076: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-15 00:37:24,079: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-15 00:37:24,080: INFO: common: Created directory at: artifacts]
[2023-08-15 00:37:24,083: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-08-15 00:37:26,330: INFO: 1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AFBA:0894:3AD00F:3CCF75:64DAAC9D
Accept-Ranges: bytes
Date: Mon, 14 Aug 2023 22:37:24 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230095-FRA
X-Cac